# Add PMIDs supporting each association from Monarch

__Can i reproduce the link+evidence in Monarch web using the api? which api+params should i use?__

- Experimental design

    1. From Monarch web browser: set an example [link + reference]
    2. Retrieve the same information via API
    
To get associations from Monarch i tested several apis. See test/{draw_edges, find_associations}.
I used biolink::association/{from/to} + exclude_evidence=true

`workdir: monarch/`

****
=Examples from Monarch=
    ~~~
    url: https://monarchinitiative.org/disease/DOID:10652#genes
    Association: Alzheimer disease (DOID:10652) - APP human gene (NCBIGene:351)
    Reference: multiple PMIDs
    
    url: https://monarchinitiative.org/disease/OMIM%3A615273#genes
    Association: NGLY1-CDDG (OMIM:615273) - NGLY1 human gene (NCBIGene:55768)
    Reference: Null
    ~~~

In [1]:
import sys, os
import requests
import json

In [60]:
# api addresses
biolink = 'https://api.monarchinitiative.org/api/association'

# write data
if not os.path.isdir(os.getcwd() + '/add_reference'): os.makedirs(os.getcwd() + '/add_reference')
    
# read data
sys.path.insert(0, './add_reference/')

# nodes
alz_dis = 'DOID:10652'
app_gene = 'NCBIGene:351'
ngly1def = 'OMIM:615273'
ngly1_gene = 'NCBIGene:55768'

#### Questions:
1. Where are the publications? in the evidence graph?
2. Can i retrieve the supporting publications shown in the web, using the api? 
3. Which api? which parameters?

## Test `/association/from` api

*Note see json formatted: https://jsonformatter.curiousconcept.com/

### Example1: Alzheimer-APP

In [28]:
# params: 'fl_excludes_evidence'
parameters = {'fl_excludes_evidence': False}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
print('number of associations: {}'.format(len(r_out.json()['associations'])))
r_out.json()

number of associations: 10


{'associations': [{'evidence_graph': {'edges': [{'obj': ':.well-known/genid/NCBIGene5664product',
      'pred': 'RO:0002205',
      'sub': 'NCBIGene:5664'},
     {'obj': 'ECO:0000322',
      'pred': 'RO:0002558',
      'sub': 'MONARCH:8aaf67cbb48a63c09423addcfc6c7db25c8f58bb'},
     {'obj': ':.well-known/genid/118105-1020VL',
      'pred': 'OBAN:association_has_subject',
      'sub': 'MONARCH:6fd03569524e67706f9b4ab9414ee94822bdb806'},
     {'obj': 'Orphanet:1020',
      'pred': 'OBAN:association_has_object',
      'sub': 'MONARCH:6fd03569524e67706f9b4ab9414ee94822bdb806'},
     {'obj': ':.well-known/genid/NCBIGene5663product',
      'pred': 'RO:0002205',
      'sub': 'NCBIGene:5663'},
     {'obj': 'Orphanet:1020',
      'pred': 'RO:0002200',
      'sub': ':.well-known/genid/121393-1020VL'},
     {'obj': 'KEGG-path:map05010',
      'pred': 'RO:0002331',
      'sub': ':.well-known/genid/NCBIGene351product'},
     {'obj': 'NCBIGene:5664',
      'pred': 'GENO:0000408',
      'sub': ':.wel

In [29]:
# params
parameters = {'fl_excludes_evidence': True}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
print('number of associations: {}'.format(len(r_out.json()['associations'])))
r_out.json()

number of associations: 10


{'associations': [{'evidence_graph': {'edges': [{'obj': ':.well-known/genid/NCBIGene5664product',
      'pred': 'RO:0002205',
      'sub': 'NCBIGene:5664'},
     {'obj': 'ECO:0000322',
      'pred': 'RO:0002558',
      'sub': 'MONARCH:8aaf67cbb48a63c09423addcfc6c7db25c8f58bb'},
     {'obj': ':.well-known/genid/118105-1020VL',
      'pred': 'OBAN:association_has_subject',
      'sub': 'MONARCH:6fd03569524e67706f9b4ab9414ee94822bdb806'},
     {'obj': 'Orphanet:1020',
      'pred': 'OBAN:association_has_object',
      'sub': 'MONARCH:6fd03569524e67706f9b4ab9414ee94822bdb806'},
     {'obj': ':.well-known/genid/NCBIGene5663product',
      'pred': 'RO:0002205',
      'sub': 'NCBIGene:5663'},
     {'obj': 'Orphanet:1020',
      'pred': 'RO:0002200',
      'sub': ':.well-known/genid/121393-1020VL'},
     {'obj': 'KEGG-path:map05010',
      'pred': 'RO:0002331',
      'sub': ':.well-known/genid/NCBIGene351product'},
     {'obj': 'NCBIGene:5664',
      'pred': 'GENO:0000408',
      'sub': ':.wel

##### RESULTS: 
- The number of associations retrieved is the same: 10
- Associations are accessible through the 'associations' key in the response dict
- response= {'associations':  [list of dict, where every dict = 1 assoc]}
- Both parameter values gave the same info content for each association, so i dont know what exactly does..
- Mainly, each assoc_dict contains (keys): id, type, sub, rel, obj, publications, evidence_graph, evidence_types, provided_by
- **Supporting publications** for each association are: {'publications': [{'id': 'PMID:111'}..{}]}, each {}= publication
- **Evidence Graph**: association is linked to the sub and obj of the relation (oban), to the evidence_types, and __publications__! (<assoc><dc:source><pmid>). {'evidence_graph': {'nodes': [{'id': 'PMID111'}..{}], 'edges': [{'sub': , 'pred': 'dc:source', 'obj': 'PMID:1111'}..{}]}}

In [30]:
# params: type of values stored in the 'publications' field? None or list()
parameters = {'fl_excludes_evidence': False}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
print('number of associations: {}'.format(len(r_out.json()['associations'])))
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])
    else:
        print(association['publications'])

number of associations: 10
None
None
None
None
None
None
None
None
None
None


In [25]:
# params: 'rows'
parameters = {'fl_excludes_evidence': False, 'rows': 4}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
print('number of associations: {}'.format(len(r_out.json()['associations'])))
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])

number of associations: 4


In [31]:
# params: 'rows'
parameters = {'fl_excludes_evidence': False, 'rows': 2000000}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
print('number of associations: {}'.format(len(r_out.json()['associations'])))
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])

number of associations: 142
PMID:13677819
PMID:17267377
PMID:7696598
PMID:22272816
PMID:22841885
PMID:20307625
PMID:9351648
PMID:7602619
PMID:23016647
PMID:9831464
PMID:22685064
PMID:10730323
PMID:10829805
PMID:23493128
PMID:3757346
PMID:21847625
PMID:15172744
PMID:12098753
PMID:15172740
PMID:12048111
PMID:12761578
PMID:3025332
PMID:9515567
PMID:14646594
PMID:10924765
PMID:6713315
PMID:20671063
PMID:8841957
PMID:2239148
PMID:17981814
PMID:15982480
PMID:12048145
PMID:12727689
PMID:16005519
PMID:16194760
PMID:21729419
PMID:15172735
PMID:15172734
PMID:20036698
PMID:22355794
PMID:21155625
PMID:17456849
PMID:19542630
PMID:22107053
PMID:19542621
PMID:16920207
PMID:12535396
PMID:23455986
PMID:16327258
PMID:12168556
PMID:15172746
PMID:21786197
PMID:9387889
PMID:8364743
PMID:21847625
PMID:9549588
PMID:23063646
PMID:12470793
PMID:12787325
PMID:1311375
PMID:10923250
PMID:11072138
PMID:14559363
PMID:11309456
PMID:8054491
PMID:23408997
PMID:19694273
PMID:8797211
PMID:22850312
PMID:22035255
PMID:123

###### RESULTS
- 'rows': number of associations in the result!!! by default it seems 'rows': 10!!!

In [32]:
# params: 'page'
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'page': 2}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
print('number of associations: {}'.format(len(r_out.json()['associations'])))
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])

number of associations: 142
PMID:13677819
PMID:17267377
PMID:7696598
PMID:22272816
PMID:22841885
PMID:20307625
PMID:9351648
PMID:7602619
PMID:23016647
PMID:9831464
PMID:22685064
PMID:10730323
PMID:10829805
PMID:23493128
PMID:3757346
PMID:21847625
PMID:15172744
PMID:12098753
PMID:15172740
PMID:12048111
PMID:12761578
PMID:3025332
PMID:9515567
PMID:14646594
PMID:10924765
PMID:6713315
PMID:20671063
PMID:8841957
PMID:2239148
PMID:17981814
PMID:15982480
PMID:12048145
PMID:12727689
PMID:16005519
PMID:16194760
PMID:21729419
PMID:15172735
PMID:15172734
PMID:20036698
PMID:22355794
PMID:21155625
PMID:17456849
PMID:19542630
PMID:22107053
PMID:19542621
PMID:16920207
PMID:12535396
PMID:23455986
PMID:16327258
PMID:12168556
PMID:15172746
PMID:21786197
PMID:9387889
PMID:8364743
PMID:21847625
PMID:9549588
PMID:23063646
PMID:12470793
PMID:12787325
PMID:1311375
PMID:10923250
PMID:11072138
PMID:14559363
PMID:11309456
PMID:8054491
PMID:23408997
PMID:19694273
PMID:8797211
PMID:22850312
PMID:22035255
PMID:123

In [33]:
# params: 'page'
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'page': 20}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
print('number of associations: {}'.format(len(r_out.json()['associations'])))
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])

number of associations: 142
PMID:13677819
PMID:17267377
PMID:7696598
PMID:22272816
PMID:22841885
PMID:20307625
PMID:9351648
PMID:7602619
PMID:23016647
PMID:9831464
PMID:22685064
PMID:10730323
PMID:10829805
PMID:23493128
PMID:3757346
PMID:21847625
PMID:15172744
PMID:12098753
PMID:15172740
PMID:12048111
PMID:12761578
PMID:3025332
PMID:9515567
PMID:14646594
PMID:10924765
PMID:6713315
PMID:20671063
PMID:8841957
PMID:2239148
PMID:17981814
PMID:15982480
PMID:12048145
PMID:12727689
PMID:16005519
PMID:16194760
PMID:21729419
PMID:15172735
PMID:15172734
PMID:20036698
PMID:22355794
PMID:21155625
PMID:17456849
PMID:19542630
PMID:22107053
PMID:19542621
PMID:16920207
PMID:12535396
PMID:23455986
PMID:16327258
PMID:12168556
PMID:15172746
PMID:21786197
PMID:9387889
PMID:8364743
PMID:21847625
PMID:9549588
PMID:23063646
PMID:12470793
PMID:12787325
PMID:1311375
PMID:10923250
PMID:11072138
PMID:14559363
PMID:11309456
PMID:8054491
PMID:23408997
PMID:19694273
PMID:8797211
PMID:22850312
PMID:22035255
PMID:123

###### RESULTS
- i don't understand what this parameter does..

In [34]:
# params: 'object_category'
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'gene'}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
print('number of associations: {}'.format(len(r_out.json()['associations'])))
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])

number of associations: 93
PMID:20610758
PMID:16651627
PMID:21157020
PMID:18583042
PMID:20111991
PMID:12572668
PMID:16204253
PMID:21980910
PMID:16325427
PMID:27117003
PMID:16492752
PMID:16969627
PMID:17239395
PMID:20640797
PMID:16266835
PMID:1671712
PMID:25352456
PMID:15591071
PMID:1497677
PMID:19818510
PMID:19286555
PMID:25714973
PMID:17430250
PMID:1678058
PMID:23827522
PMID:21209907
PMID:15993441
PMID:12746438
PMID:22507317
PMID:12852432
PMID:15590663
PMID:12192006
PMID:20157255
PMID:11470324
PMID:25352456
PMID:16651627
PMID:17573346
PMID:7596406
PMID:15622541
PMID:22507317
PMID:27117003
PMID:16449385
PMID:18227305
PMID:17962197
PMID:25714973
PMID:17192785
PMID:19374891
PMID:21460841
PMID:21460840
PMID:11709213
PMID:8346443
PMID:20819998
PMID:19734902
PMID:24162737
PMID:14517953
PMID:21715663
PMID:15750215
PMID:27117003
PMID:25352456
PMID:20157255
PMID:12852432
PMID:15750215
PMID:11144356
PMID:19374891
PMID:23022600
PMID:22944069
PMID:17409235
PMID:15750215
PMID:15732116
PMID:1850357

In [36]:
# params: 'object_category'
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'pathway'}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
print('number of associations: {}'.format(len(r_out.json()['associations'])))
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])
    else: 
        print(association['publications'])

number of associations: 37
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


###### RESULTS
- it filters by type of object

In [49]:
# Three ways to retrieve publications: by key1 'publications' or by key1 'evidence_graph' - key2 'edges' or 'nodes'
# do i retrieve the same publications from 'publications' than 'evidence_graph'?
# by publications
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'phenotype'}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
i = 0
for association in r_out.json()['associations']:
    if association['publications']:
        for publication in association['publications']:
            print(publication['id'])
            i += 1
    else: 
        print(association['publications'])
print('number of associations: {}'.format(len(r_out.json()['associations'])))
print('Number of publications: {}'.format(i))

PMID:13677819
PMID:17267377
PMID:7696598
PMID:22272816
PMID:22841885
None
PMID:20307625
PMID:9351648
PMID:7602619
PMID:23016647
PMID:9831464
PMID:22685064
PMID:10730323
PMID:10829805
PMID:23493128
PMID:3757346
PMID:21847625
PMID:15172744
PMID:12098753
PMID:15172740
PMID:12048111
PMID:12761578
PMID:3025332
PMID:9515567
PMID:14646594
PMID:10924765
None
None
None
None
None
PMID:6713315
PMID:20671063
PMID:8841957
PMID:2239148
PMID:17981814
PMID:15982480
PMID:12048145
PMID:12727689
PMID:16005519
PMID:16194760
None
None
PMID:21729419
PMID:15172735
PMID:15172734
PMID:20036698
PMID:22355794
PMID:21155625
PMID:17456849
PMID:19542630
PMID:22107053
PMID:19542621
PMID:16920207
PMID:12535396
PMID:23455986
PMID:16327258
PMID:12168556
None
PMID:15172746
PMID:21786197
PMID:9387889
PMID:8364743
PMID:21847625
PMID:9549588
PMID:23063646
PMID:12470793
PMID:12787325
PMID:1311375
None
PMID:10923250
PMID:11072138
PMID:14559363
PMID:11309456
PMID:8054491
None
None
PMID:23408997
PMID:19694273
PMID:8797211
PMID

In [50]:
# by evidence_graph::nodes
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'phenotype'}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
i = 0
for association in r_out.json()['associations']:
    for node in association['evidence_graph']['nodes']:
        if node['id'].startswith('PMID'):
            print (node['id'])
            i += 1
print('number of associations: {}'.format(len(r_out.json()['associations'])))
print('Number of publications: {}'.format(i))

PMID:22841885
PMID:13677819
PMID:7696598
PMID:22272816
PMID:17267377
PMID:23016647
PMID:20307625
PMID:9351648
PMID:7602619
PMID:9831464
PMID:10829805
PMID:3757346
PMID:10730323
PMID:22685064
PMID:23493128
PMID:15172744
PMID:12048111
PMID:15172740
PMID:21847625
PMID:12098753
PMID:9515567
PMID:14646594
PMID:3025332
PMID:12761578
PMID:10924765
PMID:20671063
PMID:6713315
PMID:2239148
PMID:17981814
PMID:8841957
PMID:16005519
PMID:12727689
PMID:16194760
PMID:12048145
PMID:15982480
PMID:15172735
PMID:20036698
PMID:15172734
PMID:21729419
PMID:22355794
PMID:22107053
PMID:19542630
PMID:17456849
PMID:19542621
PMID:21155625
PMID:16920207
PMID:12535396
PMID:12168556
PMID:16327258
PMID:23455986
PMID:21786197
PMID:9387889
PMID:8364743
PMID:15172746
PMID:21847625
PMID:1311375
PMID:9549588
PMID:12787325
PMID:23063646
PMID:12470793
PMID:11309456
PMID:8054491
PMID:11072138
PMID:14559363
PMID:10923250
PMID:22850312
PMID:8797211
PMID:22035255
PMID:19694273
PMID:23408997
PMID:20716133
PMID:8675940
PMID:1239

In [52]:
# by evidence_graph::edges
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'phenotype'}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
i = 0
for association in r_out.json()['associations']:
    for edge in association['evidence_graph']['edges']:
        if edge['obj'].startswith('PMID'):
            print (edge['obj'])
            i += 1
print('number of associations: {}'.format(len(r_out.json()['associations'])))
print('Number of publications: {}'.format(i))

PMID:13677819
PMID:17267377
PMID:7696598
PMID:22272816
PMID:22841885
PMID:20307625
PMID:9351648
PMID:7602619
PMID:23016647
PMID:9831464
PMID:22685064
PMID:10730323
PMID:10829805
PMID:23493128
PMID:3757346
PMID:21847625
PMID:15172744
PMID:12098753
PMID:15172740
PMID:12048111
PMID:12761578
PMID:3025332
PMID:9515567
PMID:14646594
PMID:10924765
PMID:6713315
PMID:20671063
PMID:8841957
PMID:2239148
PMID:17981814
PMID:15982480
PMID:12048145
PMID:12727689
PMID:16005519
PMID:16194760
PMID:21729419
PMID:15172735
PMID:15172734
PMID:20036698
PMID:22355794
PMID:21155625
PMID:17456849
PMID:19542630
PMID:22107053
PMID:19542621
PMID:16920207
PMID:12535396
PMID:23455986
PMID:16327258
PMID:12168556
PMID:15172746
PMID:21786197
PMID:9387889
PMID:8364743
PMID:21847625
PMID:9549588
PMID:23063646
PMID:12470793
PMID:12787325
PMID:1311375
PMID:10923250
PMID:11072138
PMID:14559363
PMID:11309456
PMID:8054491
PMID:23408997
PMID:19694273
PMID:8797211
PMID:22850312
PMID:22035255
PMID:12391603
PMID:16772867
PMID:867

###### RESULTS
- i retrieve the same information using the three ways
- easiest way: equivalent. 'publications' seems more direct

In [53]:
# Let's retrieve publications for alzheimer disease-app
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'gene'}
node = alz_dis
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
r_out.json()
for association in r_out.json()['associations']:
    for node in association['evidence_graph']['nodes']:
        if node['lbl'] == 'APP':
            if association['publications']:
                for publication in association['publications']:
                    print(publication['id'])
            else:
                print(association['publications'])

PMID:16651627
PMID:21157020
PMID:18583042
PMID:20111991
PMID:12572668
PMID:16204253
PMID:21980910
PMID:16325427
PMID:27117003
PMID:16492752
PMID:16969627
PMID:17239395
PMID:20640797
PMID:16266835
PMID:1671712
PMID:25352456
PMID:15591071
PMID:1497677
PMID:19818510
PMID:19286555
PMID:25714973
PMID:17430250
PMID:1678058
PMID:23827522
PMID:21209907
PMID:15993441
PMID:12746438
PMID:22507317
PMID:12852432
PMID:15590663
PMID:12192006
PMID:20157255
None


###### RESULTS
- SUCCESS!!

### Example2: NGLY deficiency-NGLY1 human gene

In [66]:
# Let's retrieve publications for NGLY1 disease-NGLY1
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'gene'}
node = ngly1def
r_out = requests.get('{}/from/{}'.format(biolink,node),params=parameters)
print(ngly1def)
r_out.json()

OMIM:615273


{'associations': [],
 'compact_associations': None,
 'facet_counts': {'object_closure': {}, 'subject_taxon_label': {}},
 'facet_pivot': None,
 'numFound': None,
 'objects': None,
 'start': None}

###### RESULTS
- NOT SUCCESS because the data is not there..

## Test `/association/to` api

### Example1: Alzheimer-APP

In [71]:
# Let's retrieve publications for alzheimer disease
# by publications
parameters = {'fl_excludes_evidence': False, 'rows': 2000000, 'object_category': 'gene'}
node = alz_dis
r_out = requests.get('{}/to/{}'.format(biolink,node),params=parameters)
r_out.json()

{'associations': [],
 'compact_associations': None,
 'facet_counts': {'object_closure': {}, 'subject_taxon_label': {}},
 'facet_pivot': None,
 'numFound': None,
 'objects': None,
 'start': None}

In [96]:
# query from gene to disease then
parameters = {'fl_excludes_evidence': False, 'rows': 2000}
node = app_gene
r_out = requests.get('{}/to/{}'.format(biolink,node),params=parameters)
r_out.json()
for association in r_out.json()['associations']:
    for node in association['evidence_graph']['nodes']:
        if node['id'] == 'DOID:10652':
            if association['publications']:
                for publication in association['publications']:
                    print(publication['id'])
            else:
                print(association['publications'])
        else:
            print('no association with the disease')

no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no assoc

no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no assoc

no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no assoc

In [94]:
# query from gene to disease then
parameters = {'fl_excludes_evidence': False, 'rows': 2000}
node = app_gene
r_out = requests.get('{}/to/{}'.format(biolink,node),params=parameters)
r_out.json()

{'associations': [{'evidence_graph': {'edges': [{'obj': 'MGI:3057146',
      'pred': 'GENO:0000382',
      'sub': ':.well-known/genid/MGI3057148U'},
     {'obj': 'NCBIGene:351', 'pred': 'GENO:0000639', 'sub': 'MGI:3057146'},
     {'obj': ':.well-known/genid/MGI3057148U',
      'pred': 'GENO:0000382',
      'sub': ':.well-known/genid/MGI2670768U-MGI3057148U'},
     {'obj': ':.well-known/genid/MGI2670768U-MGI3057148U-GENO0000649-MMRRC037554',
      'pred': 'GENO:0000222',
      'sub': 'MMRRC:037554'},
     {'obj': ':.well-known/genid/MGI2670768U-MGI3057148U',
      'pred': 'GENO:0000382',
      'sub': ':.well-known/genid/MGI2670768U-MGI3057148U-GENO0000649-MMRRC037554'}],
    'nodes': [{'id': 'MGI:3057146', 'lbl': 'Tg(PDGFB-APPSwInd)20Lms'},
     {'id': ':.well-known/genid/MGI2670768U-MGI3057148U-GENO0000649-MMRRC037554',
      'lbl': 'Mapt<tm1(EGFP)Klt>/?; Tg(PDGFB-APPSwInd)20Lms/? [n.s.]'},
     {'id': ':.well-known/genid/MGI2670768U-MGI3057148U',
      'lbl': 'Mapt<tm1(EGFP)Klt>/?; Tg

###### RESULTS
- In this api you have to reverse nodes an query from gene to disease to work
- To run the query without an error i should decrease the parameter __'rows' to 2000__
- error: 'NoneType' object is not iterable => you need to put an `if`
- No associations found

### Example2: NGLY deficiency-NGLY1 human gene

In [92]:
# Let's retrieve publications for NGLY1 disease
parameters = {'fl_excludes_evidence': False, 'rows': 2000}
node = ngly1_gene
r_out = requests.get('{}/to/{}'.format(biolink,node),params=parameters)
r_out.json()
for association in r_out.json()['associations']:
    for node in association['evidence_graph']['nodes']:
        if node['id'] == 'OMIM:615273':
            if association['publications']:
                for publication in association['publications']:
                    print(publication['id'])
            else:
                print(association['publications'])
        else:
            print('no association with the disease')

no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no association with the disease
no assoc

In [93]:
# Let's retrieve publications for NGLY1 disease
parameters = {'fl_excludes_evidence': False, 'rows': 2000}
node = ngly1_gene
r_out = requests.get('{}/to/{}'.format(biolink,node),params=parameters)
r_out.json()

{'associations': [{'evidence_graph': {'edges': [{'obj': ':.well-known/genid/610661.0004-610661.0005',
      'pred': 'GENO:0000382',
      'sub': ':.well-known/genid/genoGM25990'},
     {'obj': 'OMIM:610661.0004',
      'pred': 'GENO:0000382',
      'sub': ':.well-known/genid/610661.0004-610661.0005'},
     {'obj': ':.well-known/genid/genoGM25990',
      'pred': 'GENO:0000222',
      'sub': ':.well-known/genid/person-GM25990'},
     {'obj': 'NCBIGene:55768',
      'pred': 'GENO:0000408',
      'sub': 'OMIM:610661.0005'},
     {'obj': 'NCBIGene:55768',
      'pred': 'GENO:0000408',
      'sub': 'OMIM:610661.0004'},
     {'obj': 'OMIM:610661.0005',
      'pred': 'GENO:0000382',
      'sub': ':.well-known/genid/610661.0004-610661.0005'}],
    'nodes': [{'id': ':.well-known/genid/610661.0004-610661.0005',
      'lbl': 'NGLY1;NGLY1(3-BP DEL  1205 TTC;ARG542TER  c.1570C>T)'},
     {'id': ':.well-known/genid/person-GM25990',
      'lbl': 'affected female proband with Congenital disorder of deg

###### RESULTS
- NO results..